# Explicación de validación cruzada

**Autor:** Jazna Meza Hidalgo

**Correo Electrónico:** ymeza@ubiobio.cl

**Fecha de Creación:** Octubre de 2024  
**Versión:** 1.0  

---

## Descripción

Este notebook explica el uso de validación cruzada, considerando 2 alternativas de uso, y mostrando en cómo se puede mejorar el rendimiento.

Se trabaja un un set de datos disponible en sklearn relacionado con los tipos de flores.

## Requisitos de Software

Este notebook fue desarrollado con Python 3.9. A continuación se listan las bibliotecas necesarias:

- pandas (>=1.1.0)
- numpy (1.26.4)
- sklearn (1.3.2)

Para verificar la versión instalada ejecutar usando el nombre del paquete del cual quieres saber la versión; por ejemplo, si quieres saber la versión de sklearn usas:

```bash
import sklearn
print(sklearn.__version__)
```

In [9]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris

In [10]:
# Cargar datos Iris
iris = load_iris()
X, y = iris.data, iris.target

In [13]:
# Calcular la frecuencia de cada valor en y
unique, counts = np.unique(y, return_counts=True)
frecuencia = dict(zip(unique, counts))
frecuencia

{0: 50, 1: 50, 2: 50}



---

***Observación***

---

Se aprecia una distribución perfecta entre las 3 clases disponibles, esto es porque el conjunto de datos es académico; esta situación no es común en el mundo real.

Independiente de lo anterior, el saber que hay equilibrio de clases transforma el ***accuracy*** como una medida confiable.

# Modelo a evaluar

El modelo de `LogisticRegression` usado se puede resumir de la siguiente manera:

1. **Tipo de modelo**:
   - Es un modelo de regresión logística, que se utiliza principalmente para tareas de clasificación binaria o multiclase. En la clasificación binaria, predice la probabilidad de pertenecer a una de las dos clases.

2. **Parámetro `solver`**:
   - `solver='lbfgs'`: el algoritmo de optimización utilizado es "Limited-memory Broyden–Fletcher–Goldfarb–Shanno" (LBFGS), que es un método iterativo para encontrar el mínimo de una función. Es adecuado para problemas de clasificación con muchas características y admite la clasificación multiclase de manera eficiente.

3. **Parámetro `max_iter`**:
   - `max_iter=1000`: Indica el número máximo de iteraciones permitidas para que el algoritmo de optimización (LBFGS en este caso) converja. Un valor más alto proporciona más oportunidades para que el algoritmo encuentre la solución óptima, especialmente si los datos son complejos o tienen muchas características.

4. **Función de pérdida**:
   - Utiliza la función de pérdida logística, también conocida como "log loss" o "cross-entropy loss", para optimizar el modelo. Esto permite calcular la probabilidad de pertenecer a una clase específica.

5. **Aplicaciones**:
   - Ideal para problemas en los que la relación entre las características independientes y la probabilidad del resultado es aproximadamente lineal.

En resumen, este modelo es un clasificador de regresión logística que utiliza el algoritmo LBFGS para optimizar la función de pérdida logística, con un límite de 1000 iteraciones para la convergencia del algoritmo.

# Evaluación del modelo

## Sin validación cruzada

In [3]:
# Dividir los datos en entrenamiento (80%) y prueba (20%) sin validación cruzada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=29)

# Entrenar el modelo con los datos de entrenamiento
model = LogisticRegression(solver='lbfgs', max_iter=1000)
model.fit(X_train, y_train)

# Predecir y evaluar en los datos de prueba
y_pred = model.predict(X_test)
accuracy_normal = accuracy_score(y_test, y_pred)

print("Exactitud (sin validación cruzada):", accuracy_normal)


Exactitud (sin validación cruzada): 0.9


---

***Observación***

---


Acá se aprecia un accuracy (una de las métricas usadas en clasificación) igual al 90%.

## Con validación cruzada

Existen 2 formas de aplicar validación cruzada:

+ Usando [cross_val_score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
+ Usando [cross_validate ](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html)

#### Usando 'cros_val_score' en combinación con 'KFold'

In [25]:
# Definir el número de divisiones (split) para la validación cruzada
k_fold = KFold(n_splits=5)

# Realizar validación cruzada con 5 divisiones
accuracy_cross_val = cross_val_score(model, X_train, y_train, cv=k_fold, scoring='accuracy')

print(f"{'Exactitud por división (con validación cruzada)':<50}:", accuracy_cross_val)
print(f"{'Promedio de exactitud (con validación cruzada)':<50}: {accuracy_cross_val.mean():.4f}")


Exactitud por división (con validación cruzada)   : [1.         0.91666667 0.95833333 0.95833333 0.95833333]
Promedio de exactitud (con validación cruzada)    : 0.9583


---

***Observaciones***

---
+ Usando esta técnica de valdiación cruzada se aprecia una mejora en el rendimiento.
+ Se tienen 5 divisiones (split), significa que, en cada división, se arma un grupo que formará parte de entrenamiento y otro como validación. Gráficamente se tiene:

![Validación cruzada](https://qu4nt.github.io/sklearn-doc-es/_images/grid_search_cross_validation.png)

+ De esta forma se asegura la participación de cada observación disponible para entrenamiento en la base de validación del modelo.
+ Los cinco valores que se aprencian en el arreglo que imprime:
```
print(f"{'Exactitud por división (con validación cruzada)':<50}:", accuracy_cross_val)
```
corresponde al rendimiento obtenido por el modelo.
+ La exactitud final del modelo es el promedio de lo que se obtiene en cada división.



In [16]:
# Realizar validación cruzada con 5 divisiones sin el uso de KFold
accuracy_cross_val_x = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
print(f"{'Exactitud por división (con validación cruzada)':<50}:", accuracy_cross_val_x)
print(f"{'Promedio de exactitud (con validación cruzada)':<50}: {accuracy_cross_val_x.mean():.4f}")

Exactitud por división (con validación cruzada)   : [1.         0.91666667 1.         0.91666667 0.95833333]
Promedio de exactitud (con validación cruzada)    : 0.9583


#### Usando 'cross_validate'

##### Usando 'KFold'

In [23]:
accuracy_cross_val_full = cross_validate(model, X_train, y_train, cv=k_fold, return_train_score = True, scoring='accuracy')
print(f"{'Exactitud en los datos de validación':<55}: {accuracy_cross_val_full['test_score']}")
print(f"{'Promedio de exactitud en validación':<55}: {accuracy_cross_val_full['test_score'].mean():.4f}")
print(f"{'Exactitud en los datos de entrenamiento':<55}: {accuracy_cross_val_full['train_score']}")
print(f"{'Promedio de exactitud en entrenamiento':<55}: {accuracy_cross_val_full['train_score'].mean():.4f}")

Exactitud en los datos de validación                   : [1.         0.91666667 0.95833333 0.95833333 0.95833333]
Promedio de exactitud en validación                    : 0.9583
Exactitud en los datos de entrenamiento                : [0.97916667 0.96875    0.98958333 0.96875    0.96875   ]
Promedio de exactitud en entrenamiento                 : 0.9750


##### Sin el uso de KFold

In [21]:
accuracy_cross_val_full = cross_validate(model, X_train, y_train, cv=5, return_train_score = True, scoring='accuracy')
print(f"{'Exactitud en los datos de validación':<55}: {accuracy_cross_val_full['test_score']}")
print(f"{'Promedio de exactitud en validación':<55}: {accuracy_cross_val_full['test_score'].mean():.4f}")
print(f"{'Exactitud en los datos de entrenamiento':<55}: {accuracy_cross_val_full['train_score']}")
print(f"{'Promedio de exactitud en entrenamiento':<55}: {accuracy_cross_val_full['train_score'].mean():.4f}")

Exactitud en los datos de validación                   : [1.         0.91666667 1.         0.91666667 0.95833333]
Promedio de exactitud en validación                    : 0.9583
Exactitud en los datos de entrenamiento                : [0.97916667 0.97916667 0.96875    0.97916667 0.97916667]
Promedio de exactitud en entrenamiento                 : 0.9771



---

***¿Por qué se tienen resultados de exactitud (en entrenamiento) distintos?***

---

La diferencia en los resultados de rendimiento podría deberse a la forma en que se realiza la validación cruzada en cada caso:

1. **`cv=k_fold` vs `cv=5`**:
   - Si `k_fold` es una instancia predefinida de `KFold` o alguna otra estrategia de validación cruzada, entonces tiene un comportamiento específico en términos de cómo divide los datos. Esto incluye el número de divisiones, la aleatoriedad y si las divisiones están estratificadas o no.
   - Por otro lado, `cv=5` simplemente indica que se usará la validación cruzada con 5 ivisiones. Si el modelo es sensible a la forma en que se dividen los datos, los resultados pueden variar.

2. **Posibles factores que afectan los resultados**:
   - **Aleatoriedad en la división de los datos**: Cuando `cv=5` se usa con la opción predeterminada de `KFold`, la aleatoriedad podría ser diferente en comparación con una instancia personalizada de `KFold`.
   - **Diferente número de divisiones o configuración específica**: Si `k_fold` está configurado con diferentes parámetros, como una división estratificada (por ejemplo, `StratifiedKFold`), o una semilla de aleatoriedad distinta, los resultados pueden diferir.

En resumen, la razón principal es que el parámetro `cv` puede afectar la manera en que los datos son divididos y, por lo tanto, puede influir en la evaluación del rendimiento del modelo.


# Comentarios finales

+ ***cross_validate*** da resultados más detallados, incluyendo exactitud en entrenamiento y validación, tiempos, y permite múltiples métricas.
+ ***cross_val_score*** es una versión más simple, que sólo da la métrica de validación seleccionada para cada división.

+ Si bien es cierto que se puede aplicar validación cruzada al conjunto de datos completo, es una práctica NO RECOMENDABLE porque siempre se aconseja tener un conjunto de datos desconocidos para el modelo para ponerlo a prueba.